# Next Steps



1.   Bring in test trec data
2.   Get full rouge scores



# Walkthrough

create dataset -> https://www.youtube.com/watch?v=_l2wJb3QPdk&t=1206s 


finetune t5 for qa ->  https://www.youtube.com/watch?v=r6XY80Z9eSA 

# Libraries


In [1]:
#change runtime type to GPU
!nvidia-smi

Sat Jul 30 17:27:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install --quiet transformers==4.1.1
!pip install --quiet pytorch-lightning
!pip install --quiet tokenizers==0.9.4
!pip install sentence-transformers

     |████████████████████████████████| 1.5 MB 7.6 MB/s 
     |████████████████████████████████| 880 kB 56.9 MB/s 
     |████████████████████████████████| 2.9 MB 50.1 MB/s 
     |████████████████████████████████| 585 kB 8.0 MB/s 
     |████████████████████████████████| 596 kB 64.5 MB/s 
     |████████████████████████████████| 141 kB 74.5 MB/s 
     |████████████████████████████████| 419 kB 74.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.0 MB/s 
     |████████████████████████████████| 4.7 MB 53.0 MB/s 
     |████████████████████████████████| 1.2 MB 65.8 MB/s 
     |████████████████████████████████| 101 kB 14.3 MB/s 
     |████████████████████████████████| 6.6 MB 61.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=d890096c23c9fcaa3902585858f4d29c3931a9e82726cc83d5dfa8ca675fdaa0
  Stored in directory: /root

In [3]:
#!pip install pytorch-lightning==1.2.2

In [4]:
import time
import json
import pandas as pd
import numpy as np
import torch
from pathlib import Path

from pytorch_lightning import Trainer

from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cosine
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
import textwrap

from transformers import (
    AdamW, 
    T5ForConditionalGeneration,
    T5TokenizerFast
)

In [5]:
pl.seed_everything(15)

Global seed set to 15


15

# Data Load

In [6]:
# Download train.csv from Google Drive
#!gdown 1du0fmP8WDSW6qh7ESpugmTJ8tEWnSvMR -O medquad_train.csv
#medquad_train_df = pd.read_csv('medquad_train.csv', index_col=0)
#medquad_train_df.shape

In [7]:
!gdown 1TNcaeMAQ1egbRIV99Uk3gKZdowehkoUI -O train_df_w_context.csv
train_df_w_context = pd.read_csv('train_df_w_context.csv', index_col=0)
train_df_w_context.head()

Downloading...
From: https://drive.google.com/uc?id=1TNcaeMAQ1egbRIV99Uk3gKZdowehkoUI
To: /content/train_df_w_context.csv
100% 53.2M/53.2M [00:00<00:00, 121MB/s]


,question,context,answer
0,what is are anxiety disorders,following a healthy lifestyle can help you pre...,all of us worry about things like health money...
1,what is are von willebrand disease,wiedemannsteiner syndrome is a rare genetic co...,von willebrand disease is a bleeding disorder ...
2,how many people are affected by hypochondrogen...,melnickneedles syndrome is a rare disorder few...,hypochondrogenesis and achondrogenesis type 2 ...
3,what causes pulmonary hypertension,the mission of the national institute of neuro...,pulmonary hypertension ph begins with inflamma...
4,what are the symptoms of oculocutaneous albinism,what are the signs and symptoms of oculocutane...,what are the signs and symptoms of oculocutane...


In [8]:
# Download trec_live_data.csv from Google Drive
#change some column names to conform to medquad
!gdown 14bnH4InS57HfnT9hmimg477yFAUdDcAn -O trec_train.csv
trec_train_df = pd.read_csv('/content/trec_train.csv')
print('Training data shape: ', trec_train_df.shape)
print(trec_train_df.head(1))

Downloading...
From: https://drive.google.com/uc?id=14bnH4InS57HfnT9hmimg477yFAUdDcAn
To: /content/trec_train.csv
100% 628k/628k [00:00<00:00, 98.0MB/s]
Training data shape:  (409, 5)
   Unnamed: 0 pid type                                           question  \
0           0  Q1  NaN  Literature on Cardiac amyloidosis.  Please let...   

                                              answer  
0  Cardiac amyloidosis is a disorder caused by de...  


In [9]:
# SPLIT TREC DATASET 
trec_train_df , val_df = train_test_split(trec_train_df, test_size=0.5)
trec_val_df , trec_test_df = train_test_split(val_df, test_size=0.5)
trec_train_df.shape, trec_val_df.shape, trec_test_df.shape

((204, 5), (102, 5), (103, 5))

In [10]:
medquad_questions = train_df_w_context['question']
medquad_answers = train_df_w_context['answer']

## Embedding Based Question Retrieval for Test Questions

In [11]:
k = 10
encoder_model = SentenceTransformer('johngiorgi/declutr-base')
question_embeddings = encoder_model.encode(medquad_questions)
kmeans = KMeans(n_clusters=k)
q_clusters = kmeans.fit_predict(question_embeddings)
question_ids = {i: [] for i in range(k)}
for i, c in enumerate(q_clusters):
    question_ids[c].append(i)

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/johngiorgi_declutr-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/johngiorgi_declutr-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
def get_similar_question_context(example, answers, encoder_model, kmeans_model, 
                                 question_ids, num_neighbors):
  query_embedding = encoder_model.encode([example])
  query_cluster_dists = [cosine(query_embedding[0], kmeans_model.cluster_centers_[c])
                       for c in range(k)]
  top_clusters = np.argsort(query_cluster_dists)[:1] # top cluster
  candidate_questions = [i for c in top_clusters for i in question_ids[c]]
  candidate_question_embeds = [question_embeddings[i] for i in candidate_questions]
  knn_model = NearestNeighbors(n_neighbors=num_neighbors)
  knn_model.fit(candidate_question_embeds)
  dists, topk_idx = knn_model.kneighbors(query_embedding)
  # get answers as context
  out_str = ""
  for d, i in zip(dists[0], topk_idx[0]):
      orig_i = candidate_questions[i]
      tmp_context = answers[orig_i]
      out_str = out_str + tmp_context

  return out_str

In [13]:
test_qs = trec_test_df['question']
test_as = trec_test_df['answer']
trec_context = []

start = time.time()
for q in test_qs:
    c = get_similar_question_context(q, medquad_answers, encoder_model, kmeans, 
                                 question_ids, num_neighbors=3)
    trec_context.append(c)

print('\nTime:', time.time() - start)
print(len(trec_context))


Time: 2.3946990966796875
103


In [14]:
trec_qa_pairs = list(zip(test_qs, trec_context, test_as))
trec_test_df = pd.DataFrame(trec_qa_pairs, columns=['question', 'context', 'answer'])

# Data Ingest

In [15]:
class MedQADataset(Dataset):
  def __init__(
      self,
      data: pd.DataFrame,
      tokenizer: T5TokenizerFast,
      prefix: str='medical question: ',
      source_max_token_len: int=250,
      target_max_token_len:int=200
  ):
    self.tokenizer = tokenizer
    self.data=data
    self.prefix = prefix
    self.source_max_token_len=source_max_token_len
    self.target_max_token_len=target_max_token_len
  
  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]

    source_encoding = tokenizer(
       self.prefix + data_row["question"],
       str('context: ') + data_row["context"],
       max_length=self.source_max_token_len,
       padding="max_length",
       truncation="only_second", 
       return_attention_mask=True,
       add_special_tokens=True,
       return_tensors="pt"
    )

    target_encoding = tokenizer(
       data_row["answer"],
       max_length=self.target_max_token_len,
       padding="max_length",
       truncation=True,
       return_attention_mask=True,
       add_special_tokens=True,
       return_tensors="pt"
    )

    labels = target_encoding["input_ids"]
    labels[labels==0] = -100

    return dict(
        question=data_row["question"],
        context=data_row["context"],
        answer=data_row["answer"],
        input_ids=source_encoding["input_ids"].flatten(),
        attention_mask=source_encoding["attention_mask"].flatten(),
        labels=labels.flatten()
    )


In [16]:
MODEL_NAME = "t5-base"
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [17]:
# Train/val/test split of MedQuAD dataset
train_df , val_df = train_test_split(train_df_w_context, test_size=0.2)
val_df , test_df = train_test_split(val_df, test_size=0.1)
train_df.shape, val_df.shape, test_df.shape

((8528, 3), (1919, 3), (214, 3))

In [18]:
class MedQADataModule(pl.LightningDataModule):

  def __init__(
      self, 
      train_df: pd.DataFrame,
      val_df: pd.DataFrame,
      test_df: pd.DataFrame,
      tokenizer: T5TokenizerFast,
      prefix: str='medical question: ',
      batch_size: int = 8,
      source_max_token_len: int=250,
      target_max_token_len: int=200
  ):
    super().__init__()
    self.batch_size = batch_size
    self.train_df=train_df
    self.val_df=val_df
    self.test_df=test_df
    self.tokenizer=tokenizer
    self.prefix=prefix
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len

  def setup(self, stage=None):
    self.train_dataset = MedQADataset(
        self.train_df,
        self.tokenizer,
        self.prefix,
        self.source_max_token_len,
        self.target_max_token_len
    )
    
    self.val_dataset = MedQADataset(
        self.val_df,
        self.tokenizer,
        self.prefix,
        self.source_max_token_len,
        self.target_max_token_len
    )

    self.test_dataset = MedQADataset(
        self.test_df,
        self.tokenizer,
        self.prefix,
        self.source_max_token_len,
        self.target_max_token_len
    )

  def train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size = 4,
        num_workers = 4
    )
  def val_dataloader(self):
    return DataLoader(
        self.val_dataset,
        batch_size = 4,
        num_workers = 4
    )
  def test_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size = 4,
        num_workers = 4
    )
  


# Modeling

In [19]:
BATCH_SIZE=2
data_module = MedQADataModule(train_df, val_df, test_df, tokenizer, 
                              batch_size=BATCH_SIZE, prefix='research medical question: ')
data_module.setup()

In [20]:
class MedicalQAModel(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)

  def forward(self, input_ids, attention_mask, labels=None):#make labels optional as they will not be included when testing
    output = self.model(
        input_ids = input_ids,
        attention_mask = attention_mask,
        labels=labels
    )
    return output.loss, output.logits

  # TODO look into pytorch lightning
  # Uses data module
  def training_step(self, batch, batch_index):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels=batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return loss
  def validation_step(self, batch, batch_index):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels=batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss
  def test_step(self, batch, batch_index):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels=batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  def configure_optimizers(self):
    #lr based on proposed value from original paper
    return AdamW(self.parameters(), lr = 0.0001)

# Training

In [21]:
#connect to drive to save checkpoints
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
# #create checkpoint call back to save the best model during trainig
checkpoint_callback = ModelCheckpoint(
    dirpath="/content/drive/My Drive/checkpoints",
    filename="{epoch:02d}-{val_loss:.4f}",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

In [23]:
logger = TensorBoardLogger("training-logs", name="med-qa")

trainer = Trainer(
    logger=logger,
    callbacks=[checkpoint_callback],
    max_epochs=20,
    gpus=1,
    progress_bar_refresh_rate=30
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [24]:
# Instantiate model object and train
# model = MedicalQAModel()
## or from checkpoint 
#last_checkpoint = f"/content/drive/MyDrive/checkpoints/best-checkpoint-epoch=02-val_loss=1.7679.ckpt"
#model = MedicalQAModel.load_from_checkpoint(last_checkpoint)

In [25]:
# trainer.fit(model, data_module)

# Evaluate

In [22]:
# Load from checkpoint 
best_checkpoint = f"/content/drive/MyDrive/checkpoints/epoch=11-val_loss=1.7662.ckpt"
best_model = MedicalQAModel.load_from_checkpoint(best_checkpoint)
best_model.freeze()

In [23]:
!pip install  --quiet datasets nltk
import numpy as np
import torch
from torchmetrics.text.rouge import ROUGEScore
rouge = ROUGEScore()
import nltk
nltk.download('punkt')

     |████████████████████████████████| 365 kB 9.7 MB/s 
     |████████████████████████████████| 212 kB 70.5 MB/s 
     |████████████████████████████████| 127 kB 71.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [24]:
def generate_answer(question, model):
  source_encoding = tokenizer(
      str('medical question: ') + question["question"],
      str('context: ') + question["context"], # including context
      max_length=150,
      padding="max_length",
      #TODO: why does truncation not work?
      truncation=True, 
      return_attention_mask=True,
      return_tensors="pt"
  )

  generated_ids = model.model.generate(
      input_ids = source_encoding["input_ids"],
      attention_mask = source_encoding["attention_mask"],
      num_beams= 2, # 1 = greedy search
      max_length=200,
      repetition_penalty=1.5,
      early_stopping=True,
      use_cache=True
  )

  preds = [
           tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenizer_spaces=True)
           for generated_id in generated_ids
  ]

  decoded_preds = "".join(preds)

  # Rouge expects a newline after each sentence
  formatted_pred = "\n".join(nltk.sent_tokenize(decoded_preds))

  return formatted_pred

In [25]:
trec_test_df.shape

(103, 3)

In [26]:
all_predictions=[]
for q in range(trec_test_df.shape[0]):
  print(f'Generating answer for test question {q}')
  question = trec_test_df.iloc[q]
  test_pred = generate_answer(question, best_model)
  all_predictions.append(test_pred)

Generating answer for test question 0
Generating answer for test question 1
Generating answer for test question 2
Generating answer for test question 3
Generating answer for test question 4
Generating answer for test question 5
Generating answer for test question 6
Generating answer for test question 7
Generating answer for test question 8
Generating answer for test question 9
Generating answer for test question 10
Generating answer for test question 11
Generating answer for test question 12
Generating answer for test question 13
Generating answer for test question 14
Generating answer for test question 15
Generating answer for test question 16
Generating answer for test question 17
Generating answer for test question 18
Generating answer for test question 19
Generating answer for test question 20
Generating answer for test question 21
Generating answer for test question 22
Generating answer for test question 23
Generating answer for test question 24
Generating answer for test question

In [27]:
len(all_predictions)

103

In [28]:
all_answers = trec_test_df['answer']
formatted_labels = ["\n".join(nltk.sent_tokenize(label)) 
                  for label in all_answers]

In [29]:
preds = pd.DataFrame()
preds['questions'] = trec_test_df["question"]
preds['context'] = trec_test_df['context']
preds['preds'] = all_predictions
preds['labels'] = formatted_labels
preds

,questions,context,preds,labels
0,I had Open RNYgastric bypass surgery.I have pr...,what are the signs and symptoms of brachydacty...,what are the signs and symptoms of brachydacty...,Follow Your Diet Carefully After gastric bypas...
1,hi doctor my name is [NAME] m from [LOCATION] ...,kuskokwim disease is a congenital present at b...,kuskokwim disease is a congenital present at b...,Treatment depends on the source of the problem...
2,I LIKE TO KNOW RECENT THERAPY ON ARRHYTHMIA OF...,what are the signs and symptoms of patterned d...,what are the signs and symptoms of patterned d...,Treatment to restore a normal heart rhythm may...
3,I think someone drugged my alcoholic beverage ...,diabetes is a very serious disease over time d...,diabetes is a very serious disease over time d...,These drugs are very powerful.\nThey can affec...
4,Not sure if I need to see my doctor. Hope you ...,frequently asked questions faqs cystic echinoc...,what are the signs and symptoms of spondylomet...,"See ""How do I know when I am working out too h..."
...,...,...,...,...
98,I just read an article submitted by you that s...,in the united states infestation with head lic...,in the united states infestation with head lic...,If the underlying cause of burning mouth syndr...
99,I am looking for some information about sympto...,methylmalonic acidemia with homocystinuria can...,methylmalonic acidemia with homocystinuria can...,Adult ADHD symptoms may include: Trouble focu...
100,Customer Service Request. My daughter has Tur...,praderwilli syndrome is caused by the loss of ...,praderwilli syndrome is caused by the loss of ...,Most cases of Turner syndrome are not inherite...
101,we need treatment for balance disorder. my fat...,diabetes is a very serious disease over time d...,diabetes is a very serious disease over time d...,How are balance disorders treated?\nThe first ...


In [30]:
# compute metrics
predictions_labels_and_answers = zip(all_predictions, formatted_labels)

rouges = [rouge(p_a[0], p_a[1]) for p_a in predictions_labels_and_answers]

In [31]:
rouges_df = pd.DataFrame(rouges)
print('___________ Rouge 1 ___________')
print(f'f1: {(rouges_df["rouge1_fmeasure"].mean()* 100):.2f}')
print(f'precision: {(rouges_df["rouge1_precision"].mean()* 100):.2f}')
print(f'recall: {(rouges_df["rouge1_recall"].mean()* 100):.2f}')
print('___________ Rouge 2 ___________')
print(f'f1: {(rouges_df["rouge2_fmeasure"].mean()* 100):.2f}')
print(f'precision: {(rouges_df["rouge2_precision"].mean()* 100):.2f}')
print(f'recall: {(rouges_df["rouge2_recall"].mean()* 100):.2f}')
print('___________ Rouge L ___________')
print(f'f1: {(rouges_df["rougeL_fmeasure"].mean()* 100):.2f}')
print(f'precision: {(rouges_df["rougeL_precision"].mean()* 100):.2f}')
print(f'recall: {(rouges_df["rougeL_recall"].mean()* 100):.2f}')
print('___________ Rouge Lsum ________')
print(f'f1: {(rouges_df["rougeLsum_fmeasure"].mean()* 100):.2f}')
print(f'precision: {(rouges_df["rougeLsum_precision"].mean()* 100):.2f}')
print(f'recall: {(rouges_df["rougeLsum_recall"].mean()* 100):.2f}')

___________ Rouge 1 ___________
f1: 13.67
precision: 20.53
recall: 15.45
___________ Rouge 2 ___________
f1: 1.04
precision: 1.94
recall: 1.11
___________ Rouge L ___________
f1: 8.68
precision: 12.78
recall: 10.35
___________ Rouge Lsum ________
f1: 12.00
precision: 18.37
recall: 13.40
